Checking DNS pkt ratios in benign and malicious pcaps


## Importing Scapy 

In [14]:
!pip install scapy
from scapy.utils import RawPcapReader, rdpcap
from scapy.layers.l2 import Ether
from scapy.layers.dns import DNS, DNSQR
from scapy.layers.inet import IP, TCP, UDP

Mount G Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


File list for parsing

In [21]:
with open('/content/drive/My Drive/Botnet Dataset/shreyyash/pcap_file_list.txt', 'r') as pcap_list:
    data = pcap_list.read()
    print (f'Following files will be parsed\n{data}')

Following files will be parsed
/content/drive/My Drive/Botnet Dataset/CTU DATASET/CTU MIXED CAPTURES/CTU_Mixed_Capture_1/2015-07-28_mixed.pcap
/content/drive/My Drive/Botnet Dataset/CTU DATASET/CTU MIXED CAPTURES/CTU_Mixed_Capture_1/2015-07-28_mixed.before.infection.pcap


Add file to parse list

In [19]:
with open('/content/drive/My Drive/Botnet Dataset/shreyyash/pcap_file_list.txt', 'a') as pcap_list:
    new_file_path = str(input('Enter path of file to add to list: '))
    new_file_path += '\n' + new_file_path
    pcap_list.write(new_file_path)


Enter path of file to add to list: test2
/content/drive/My Drive/Botnet Dataset/CTU DATASET/CTU MIXED CAPTURES/CTU_Mixed_Capture_1/2015-07-28_mixed.pcap
/content/drive/My Drive/Botnet Dataset/CTU DATASET/CTU MIXED CAPTURES/CTU_Mixed_Capture_1/2015-07-28_mixed.before.infection.pcaptest1test2


Fn to open txt file and pass pcap file names

In [0]:
def pcap_name_read():
    with open('/content/drive/My Drive/Botnet Dataset/shreyyash/pcap_file_list.txt', 'r') as pcap_list:
        data = pcap_list.read()
        pcap_names = data.split('\n')
        
    for file_name in pcap_names:
        pcap_streams(file_name)
        pcap_pkt_data(file_name)

Fn to make uni directional flows, count DNS flows and show %s using rdpcap

In [0]:
def pcap_streams(file_name):
    pcap_data = rdpcap(file_name)
    print (f'{file_name} read')

    pcap_streams = pcap_data.sessions()    # extracting flows (Unidirectiuonal)

    #Counting DNS Flows from total flows
    dns_count = 0
    dnsPortNo_matchPattern = ':53 '
    for stream_name in pcap_streams.keys():
        if dnsPortNo_matchPattern in (stream_name + " "):
            dns_count += 1
 
    print (f'{dns_count} DNS Flows and {len (pcap_streams)} Total flows in PCAP file')
    print (f'Ratio of DNS flows to Total is {(dns_count*100)/(len (pcap_streams))}%')

Fn to count DNS pkts and show %s using RawPcapReader

In [0]:
def pcap_pkt_data(file_name):
    #DNS pkt count in total pkts
    i = 0   #pkt count
    dns_pkt_count = 0

    for (pkt_data, pkt_metadata ) in RawPcapReader(file_name):
        frm = Ether(pkt_data)
        i += 1
        try:
            if DNS in frm and frm[IP].proto == 17 and((frm[UDP].sport == 53) or (frm[UDP].dport == 53)):         
                dns_pkt_count += 1
        except:
            print (f'check pkt no {i}')

    print(f'{dns_pkt_count}  DNS pkts from total {i} pkts, ratio is {(dns_pkt_count*100)/(i)}%\n')

Main Fn of file

In [0]:
pcap_name_read()